## Deferred Initialization

There’s no way Pytorch(or any other framework for that matter) could predict what the input dimensionality of a network would be. Later on, when working with convolutional networks and images this problem will become even more pertinent, since the input dimensionality (i.e. the resolution of an image) will affect the dimensionality of subsequent layers at a long range. Hence, <font color=red>the ability to set parameters without the need to know at the time of writing the code what the dimensionality</font> is can greatly simplify statistical modeling. In what follows, we will discuss how this works using initialization as an example. After all, we cannot initialize variables that we don’t know exist.

## Instantiating a Network

In [1]:
import torch
import torch.nn as nn

In [14]:
def getnet(in_features,out_features):
    net=nn.Sequential(
    nn.Linear(in_features,256),
    nn.ReLU(),
    nn.Linear(256,out_features))
    
    return net

net = getnet(20,10)

The in_features can be not specified clearly when defining the model, but those paramters need to be specified clearly when creating an instance. 
### In pytorch it is not possible to define a layer without mentioning the in_features for that layer

In [9]:
for name, param in net.named_parameters():
    print(name, param.shape)
    
# # equivalent form
# for name, param in net.state_dict().items():
#     print(name, param.shape)

0.weight torch.Size([256, 20])
0.bias torch.Size([256])
2.weight torch.Size([10, 256])
2.bias torch.Size([10])


Now it is possible to make a network learn the size from a input data by making a custom nn module using pytorch

 ## Deferred Initialization in Practice
Now that we know how it works in theory, let’s see when the initialization is actually triggered. In order to do so, we mock
up an initializer.The initializer **init_weights** when evoked it initializes the weight of the network.It also sets the weights of the neural network to a non zero value which helps as neural networks tend to get stuck in local minima, so it's a good idea to give them many different starting values. You can't do that if they all start at zero.


In [15]:
def init_weights(m):
    print("Init:", m)
# the init is conducted in a way of bottom-up
    
net.apply(init_weights)

print(net[0].weight.size())
print(net[2].weight.size())

Init: Linear(in_features=20, out_features=256, bias=True)
Init: ReLU()
Init: Linear(in_features=256, out_features=10, bias=True)
Init: Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([256, 20])
torch.Size([10, 256])


In [11]:
x = torch.rand((2,20))
y = net(x) # Forward computation
for name, param in net.named_parameters():
    print(name, param.shape)

0.weight torch.Size([256, 20])
0.bias torch.Size([256])
2.weight torch.Size([10, 256])
2.bias torch.Size([10])


<font color=red> Note that for the linear layer, the weight size = `out_features, in_features`, i.e., the data compuation is $\mathbf{W}\mathbf{x} + \mathbf{b}$. </font> 

The main difference to before is that as soon as we knew the input dimensionality, x is R 20 it was possible to define the weight matrix for the first layer, i.e. W1 is R 256 * 20. With that out of the way, we can progress to the second layer, define its dimensionality to be 10 * 256 and so on through the computational graph and bind all the dimensions as they become available. Once this is known, we can proceed by initializing parameters.

As mentioned at the beginning of this section, deferred initialization can also cause confusion. Before the first forward
calculation, we were unable to directly manipulate the model parameters, for example, we could not use the data and
set_data functions to get and modify the parameters. Therefore, we often force initialization by sending a sample
observation through the network.

## Forced Initialization

Deferred initialization does not occur if the system knows the shape of all parameters when calling the initialize
function. This can occur in two cases:
1. We’ve already seen some data and we just want to reset the parameters.
2. We specified all input and output dimensions of the network when defining it.


The first case works just fine, as illustrated below.

once we see some data and define the parameters and after that we initialize those parameters again using **init_weights** function

In [12]:
net1=nn.Sequential()
net1.add_module("Linear1",nn.Linear(20,256))
net1.add_module("Linear2",nn.Linear(256,10))
print(net1)

Sequential(
  (Linear1): Linear(in_features=20, out_features=256, bias=True)
  (Linear2): Linear(in_features=256, out_features=10, bias=True)
)


In [13]:
def init_weights_forced(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)

In second case we specify the remaining set of parameters when initializing the network

In [14]:
net1.apply(init_weights_forced)
for name,param in net1.named_parameters():
    print(name,param.shape)

Linear1.weight torch.Size([256, 20])
Linear1.bias torch.Size([256])
Linear2.weight torch.Size([10, 256])
Linear2.bias torch.Size([10])


In the above case we have the data before hand and now we use that datato allow the model itself the way it wants to set the  in_features and the parameters.

The second case requires us to specify the remaining set of parameters when creating the layer. For instance, for dense
layers we also need to specify the in_features so that initialization can occur immediately once  called.

## Summary

1.  Pytorch doesnot provide a inbuilt feature for deferred initialization.
2.  <font color=red>Deferred initialization is a good thing. It allows  to set many things automagically and it removes a great source of errors from defining novel network architectures.</font>
3.  We can override this by specifying all implicitly defined variables.


## Exercises

1. What happens if you specify only parts of the input dimensions. Do you still get immediate initialization?
2. What happens if you specify mismatching dimensions?
3. What would you need to do if you have input of varying dimensionality? Hint - look at parameter tying.